大体思路：先将数据中的代码与日期作为索引，并且将它们转化为与待填充数据表相同的格式，然后依据填充表筛选数据。将相同年份的值分组，通过自定义函数计算收入熵。最后连接到填充表

In [1]:
import pandas as pd
from math import log10
df1 = pd.read_csv('company.csv')
df2 = pd.read_csv('company_data.csv',index_col=['证券代码','日期'])

In [2]:
df2=df2.rename(index=lambda x:f'#{x:0>6}',level=0)
df2=df2.rename(index=lambda x:int(x[:4]),level=1)

In [3]:
need_data = [tuple(xi) for xi in df1.values] 
df_need=df2.loc[df2.index.isin(need_data)]
df_need.head()

收入类型          收入额
证券代码    日期                     
#000007 2014     1  61461724.73
        2014     2         0.00
        2014     3  61461724.73
        2014     4  98631112.52
        2014     5  80000000.00

In [4]:
def entropy(x):
    #收入熵函数，因为有sum计算，使用lambda 表达式较慢
    s=x.sum()
    p=[xi/s for xi in x if xi>0]
    return sum([pi*log10(pi) for pi in p])

In [5]:
df_entropy=pd.DataFrame(df_need.groupby(['证券代码','日期'])['收入额'].agg([('收入熵',entropy)]))
df_entropy=df_entropy.reset_index()

In [6]:
df1.merge(df_entropy,how='left',on=['证券代码','日期'])

,证券代码,日期,收入熵
0,#000007,2014,-1.333485
1,#000403,2015,-1.211936
2,#000408,2016,-1.224077
3,#000408,2017,NaN
4,#000426,2015,-1.339480
...,...,...,...
1043,#600978,2011,-1.441449
1044,#600978,2014,-1.210856
1045,#600978,2015,-1.308368
1046,#600978,2016,-1.312072
